In [1]:
import gzip
import shutil
from struct import unpack
from collections import namedtuple, Counter, defaultdict
from pathlib import Path
from urllib.request import urlretrieve
from urllib.parse import urljoin
from datetime import timedelta
from time import time

import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns

In [2]:
def format_time(time):
  """ Returns a time with format HH:MM:SS from numeric time value, in seconds """
  m, s = divmod(time, 60)
  h, m = divmod(m, 60)
  return f"{h:0>2.0f}:{m:0>2.0f}:{s:0>5.2f}"

# ITCH

ITCH, shorthand for TotalView-ITCH, is Nasdaq's direct data-feed reporting protocol. It allows subscribers to track individual orders from placement to execution or cancellation. With proper parsing, it allows reconstruction of the order book. Being able to read ITCH data is invaluable for algorithmic trading.

So that's what we are going to work on today.

Check the TotalView-ITCH documentation here:
https://www.nasdaqtrader.com/content/technicalsupport/specifications/dataproducts/NQTVITCHSpecification.pdf

#### Set Data Paths

In [3]:
data_path = Path('data')
itch_store = str(data_path / 'itch.h5')
order_book_store = data_path / 'order_book.h5'

In [4]:
FTP_URL = 'ftp://emi.nasdaq.com/ITCH/'
SOURCE_FILE = '10302019.NASDAQ_ITCH50.gz'

#### Download and Unzip

In [5]:
def download(url):
  """ Download and unzip ITCH data, if available """

  # create directory if we need to
  if not data_path.exists():
    data_path.mkdir()
    print('Directory Created')
  else:
    print('Directory Already Exists')

  filename = data_path / url.split('/')[-1]

  # Download file if need be
  if not filename.exists():
    print('Downloading file...', url)
    urlretrieve(url, filename)
  else:
    print('File already downloaded')

  unzipped = data_path / (filename.stem + '.bin')

  # unzip feel if need be
  if not unzipped.exists():
    print('Unzipping to', unzipped)
    with gzip.open(str(filename), 'rb') as f_in:
      with open(unzipped, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
  else:
    print('File already unzipped')
  
  return unzipped

In [6]:
file_name = download(urljoin(FTP_URL, SOURCE_FILE))
date = file_name.name.split('.')[0]

Directory Created
Unzipping to data/10302019.NASDAQ_ITCH50.bin


## Processing the file

The ITCH data is in a binary file, but thankfully Python has the `struct` module to help us parse the information. 

I highly recommend taking a look at the module documentation here: 
https://docs.python.org/3/library/struct.html

### Creating Formatting Dictionaries

We will use a few dictionaries to "translate" the ITCH data into something we can more easily read and understand.

In [7]:
event_codes = {'O':'Start of Messages',
               'S':'Start of System Hours',
               'Q':'Start of Market Hours',
               'M':'End of Market Hours',
               'E':'End of System Hours',
               'C':'End of Messages'}

In [8]:
variable_encoding = {'primary_market_maker': {'Y':1, 'N':0},
                     'printable': {'Y':1, 'N':0},
                     'buy_sell_indicator': {'B':1, 'S':-1},
                     'cross_type': {'O':0, 'C':1, 'H':2},
                     'imbalance_direction': {'B':0, 'S':1, 'N':0, 'O':-1}}

In [9]:
formatting_dict = {
    ('integer', 2):'H',
    ('integer', 4):'I',
    ('integer', 6):'6s',
    ('integer', 8):'Q',

    ('alpha', 1):'s',
    ('alpha', 2):'2s',
    ('alpha', 4):'4s',
    ('alpha', 8):'8s',

    ('price_4', 4):'I',
    ('price_8', 8):'Q'
}

### Message Types

There are many different message types in ITCH, and the protocol includes an encoding system for the different types. It is a pretty long list...but thankfully, Stefan Jansen or one of his collaborators has taken on the tedious task of putting the codes into a .xlsx file for us. 

Get the file here:
https://github.com/stefan-jansen/machine-learning-for-trading/blob/master/02_market_and_fundamental_data/01_NASDAQ_TotalView-ITCH_Order_Book/message_types.xlsx



In [16]:
message_types = pd.read_excel('message_types.xlsx', sheet_name='messages')
message_types = message_types.sort_values('id').drop('id', axis=1)
message_types.head()

,Name,Offset,Length,Value,Notes
0,Message Type,0,1,S,System Event Message
1,Stock Locate,1,2,Integer,Always 0
2,Tracking Number,3,2,Integer,Nasdaq internal tracking number
3,Timestamp,5,6,Integer,Nanoseconds since midnight
4,Event Code,11,1,Alpha,See System Event Codes below


#### Clean the message_types Data

In [11]:
def clean_message_types(data):
  """ Cleans up the message_types DataFrame, making it easier to work with in Python """
  data.columns = [c.lower().strip() for c in data.columns]
  data.value = data.value.str.strip()
  data.name = (data.name.str.strip().str.lower().str.replace(' ', '_').str.replace('-', '_').str.replace('/', '_'))
  data.notes = data.notes.str.strip()
  data['message_type'] = data.loc[data.name == 'message_type', 'value']
  
  return data

In [17]:
message_types = clean_message_types(message_types)

In [18]:
message_types.head()

,name,offset,length,value,notes,message_type
0,message_type,0,1,S,System Event Message,S
1,stock_locate,1,2,Integer,Always 0,NaN
2,tracking_number,3,2,Integer,Nasdaq internal tracking number,NaN
3,timestamp,5,6,Integer,Nanoseconds since midnight,NaN
4,event_code,11,1,Alpha,See System Event Codes below,NaN


We are also going to extract the message_types so we can easily replace them with message_labels to make it more readable.

In [19]:
message_labels = message_types.loc[:, ['message_type', 'notes']].dropna()
message_labels.columns = ['message_type', 'name']
message_labels.name = message_labels.name.str.lower().str.replace('message', '').str.replace('.', '').str.strip().str.replace(' ', '_')
message_labels.head()

,message_type,name
0,S,system_event
5,R,stock_directory
23,H,stock_trading_action
31,Y,reg_sho_short_sale_price_test_restricted_indic...
37,L,market_participant_position


### Finalize Message Types

In [20]:
message_types['message_type'] = message_types['message_type'].ffill()
message_types = message_types[message_types['name'] != 'message_type']
message_types['value'] = message_types['value'].str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
message_types.head()

,name,offset,length,value,notes,message_type
1,stock_locate,1,2,integer,Always 0,S
2,tracking_number,3,2,integer,Nasdaq internal tracking number,S
3,timestamp,5,6,integer,Nanoseconds since midnight,S
4,event_code,11,1,alpha,See System Event Codes below,S
6,stock_locate,1,2,integer,Locate Code uniquely assigned to the security ...,R


In [21]:
message_types.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 152 entries, 1 to 172
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name          152 non-null    object
 1   offset        152 non-null    int64 
 2   length        152 non-null    int64 
 3   value         152 non-null    object
 4   notes         152 non-null    object
 5   message_type  152 non-null    object
dtypes: int64(2), object(4)
memory usage: 8.3+ KB


### Save Cleaned message_types DataFrame to .csv file

In [22]:
message_types.to_csv('clean_message_types.csv', index=False)

In [23]:
message_types = pd.read_csv('clean_message_types.csv')

## Use message_types to format ITCH data!

In [26]:
# create format tuples
format_tuples = message_types[['value', 'length']].apply(tuple, axis=1).map(formatting_dict)
format_tuples

0       H
1       H
2      6s
3       s
4       H
       ..
147     I
148     I
149     I
150     s
151     s
Length: 152, dtype: object

In [33]:
message_types['format'] = format_tuples
alphas = message_types[message_types['value'] == 'alpha']
alphas = alphas.set_index('name')
alphas_grouped = alphas.groupby('message_type')
alphas_formats = {k:v.to_dict() for k, v in alphas_grouped['format']}
alphas_lengths = {k:v.add(5).to_dict() for k, v in alphas_grouped['length']}

In [35]:
# create message classes
message_classes, fstrings = {}, {}
for t, message in message_types.groupby('message_type'):
  message_classes[t] = namedtuple(typename=t, field_names=message['name'].tolist())
  fstrings[t] = '>' + ''.join(message['format'].tolist())

In [39]:
alphas.head()

,offset,length,value,notes,message_type,format
name,,,,,,
event_code,11,1,alpha,See System Event Codes below,S,s
stock,11,8,alpha,Denotes the security symbol for the issue in t...,R,8s
market_category,19,1,alpha,Indicates Listing market or listing market tie...,R,s
financial_status_indicator,20,1,alpha,"For Nasdaq listed issues, this field indicates...",R,s
round_lots_only,25,1,alpha,Indicates if Nasdaq system limits order entry ...,R,s


In [43]:
# post-processing function for alphanumeric fields
def format_alpha(mtype, data):
  """ processes bytes of alphanumeric data (type alpha) """
  for col in alpha_formats.get(mtype).keys():
    if mtype != 'R' and col == 'stock':
      data = data.drop(col, axis=1)
      continue
    data.loc[:, col] = data.loc[:, col].str.decode('utf-8').str.strip()
    
    if encoding.get(col):
      data.loc[:, col] = data.loc[:, col].map(encoding.get(col))

  return data

## Get messages from Binary file!

In [44]:
def get_messages(m):
  """ get the messages from the binary file! """
  # open file
  with pd.HDFStore(itch_store) as store:
    # cycle through messages
    for mtype, data in m.items():
      # change data to DataFrame
      data = pd.DataFrame(data)

      # parse timestamps
      data.timestamp = data.timestamp.apply(int.from_bytes, byteorder='big')
      data.timestamp = pd.to_timedelta(data.timestep)

      # format_alpha
      if mtype in alphas_formats.keys():
        data = format_alpha(mtype, data)

      s = alphas_lengths.get(mtype)

      if s:
        s = {c:s.get(c) for c in data.columns}
      
      dc = ['stock_locate']

      if m == 'R':
        dc.append('stock')

      try:
        store.append(mtype, data, format='t', min_itemsize=s, data_columns=dc)

      except Exception as e:
        print(e)
        print(mtype)
        print(data.info())
        print(pd.Series(list(m.keys())).value_counts())
        data.to_csv('data.csv', index=False)
        return 1
  return 0